# Pull Data from AWS

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [45.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-14 14:28:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.09MB/s    in 0.2s    

2021-02-14 14:28:42 (6.09 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import pandas as pd
from pyspark import SparkFiles

In [5]:
# Read in Zillow data from S3 Buckets
url = "https://housingdata21.s3.us-east-2.amazonaws.com/zhvi_2018.csv"
spark.sparkContext.addFile(url)
zillow_df = spark.read.csv(SparkFiles.get("zhvi_2018.csv"), sep=",", header=True, inferSchema=True)
zillow_df.show()

+-------+----------+
|zipcode|      zhvi|
+-------+----------+
|  60002|  203155.5|
|  60004| 371788.75|
|  60005| 375617.67|
|  60007|  290179.5|
|  60008| 257214.25|
|  60010|  503737.0|
|  60012| 311768.92|
|  60013| 237906.58|
|  60014| 232307.17|
|  60015| 489269.25|
|  60016| 285147.67|
|  60018| 274593.83|
|  60020| 165690.42|
|  60021| 243928.17|
|  60022|1037845.67|
|  60025| 530058.75|
|  60026| 613297.75|
|  60029| 812012.75|
|  60030| 219686.75|
|  60031|  243986.5|
+-------+----------+
only showing top 20 rows



In [6]:
# Read in income data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/MedianIncome_Zip.csv"
spark.sparkContext.addFile(url)
income_df = spark.read.csv(SparkFiles.get("MedianIncome_Zip.csv"), sep=",", header=True, inferSchema=True)
income_df.show()

+--------+-----------------------+
|Zipcodes|Median income (dollars)|
+--------+-----------------------+
|   52761|                  55132|
|   60002|                  82692|
|   60004|                  94096|
|   60005|                  78109|
|   60007|                  79590|
|   60008|                  66462|
|   60010|                 137994|
|   60012|                 127651|
|   60013|                  98377|
|   60014|                  87256|
|   60015|                 142409|
|   60016|                  64444|
|   60018|                  60751|
|   60020|                  57518|
|   60021|                 106960|
|   60022|                 209821|
|   60025|                  97881|
|   60026|                 111875|
|   60029|                 200500|
|   60030|                  87449|
+--------+-----------------------+
only showing top 20 rows



In [7]:
# Read in ACS data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/acs_data.csv"
spark.sparkContext.addFile(url)
acs_df = spark.read.csv(SparkFiles.get("acs_data.csv"), sep=",", header=True, inferSchema=True)
acs_df.show()

+-------+---------+-----+------+-------+------+-------+--------+-------+--------+--------+--------+-----------+
|zipcode|total_pop| male|female|med_age|race_w|race_aa|race_nat|race_as|race_api|race_oth|eth_hisp|eth_nonhisp|
+-------+---------+-----+------+-------+------+-------+--------+-------+--------+--------+--------+-----------+
|  52761|    30619|15205| 15414|   37.8| 28856|   1504|     255|    493|      11|     603|    5480|      25139|
|  60002|    24066|12076| 11990|   43.3| 22792|    869|      89|    630|      31|     402|    1662|      22404|
|  60004|    50915|24193| 26722|     43| 43996|    738|      79|   5458|     116|    1597|    4013|      46902|
|  60005|    29539|15040| 14499|   43.8| 24937|    892|     143|   2961|       0|    1089|    3163|      26376|
|  60007|    33420|16504| 16916|   43.8| 28444|    532|     245|   3680|      24|    1500|    4020|      29400|
|  60008|    21833|10604| 11229|   37.4| 15923|   1179|     325|   1992|      92|    2976|    5430|     

In [8]:
# Read in crime data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/crime_data.csv"
spark.sparkContext.addFile(url)
crime_df = spark.read.csv(SparkFiles.get("crime_data.csv"), sep=",", header=True, inferSchema=True)
crime_df.show()

+-----------------+---------+------------+----+-------+-----------+----------+--------+---------+---------+-----+
|              _c0|vio_crime|murder_mansl|rape|robbery|agg_assault|prop_crime|burglary|lar_theft|car_theft|arson|
+-----------------+---------+------------+----+-------+-----------+----------+--------+---------+---------+-----+
|          Addison|       61|           1|  18|      6|         36|       464|      65|      361|       38|    4|
|           Albany|        1|           0|   1|      0|          0|         6|       2|        4|        0|    0|
|            Aledo|        0|           0|   0|      0|          0|        26|       3|       23|        0|    0|
|           Alexis|        0|           0|   0|      0|          0|        15|       7|        4|        4|    0|
|        Algonquin|       27|           0|   7|      4|         16|       265|      18|      243|        4|    0|
|            Alsip|       24|           2|   0|     16|          6|       427|      66| 

In [9]:
# Read in city data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/zipcode_to_city.csv"
spark.sparkContext.addFile(url)
city_df = spark.read.csv(SparkFiles.get("zipcode_to_city.csv"), sep=",", header=True, inferSchema=True)
city_df.show()

+--------+-----------------+
|Zip Code|             City|
+--------+-----------------+
|   60001|            Alden|
|   60002|          Antioch|
|   60004|Arlington Heights|
|   60005|Arlington Heights|
|   60006|Arlington Heights|
|   60007|Elk Grove Village|
|   60008|  Rolling Meadows|
|   60009|Elk Grove Village|
|   60010|       Barrington|
|   60011|       Barrington|
|   60012|     Crystal Lake|
|   60013|             Cary|
|   60014|     Crystal Lake|
|   60015|        Deerfield|
|   60016|      Des Plaines|
|   60017|      Des Plaines|
|   60018|      Des Plaines|
|   60019|      Des Plaines|
|   60020|         Fox Lake|
|   60021|  Fox River Grove|
+--------+-----------------+
only showing top 20 rows



In [10]:
# Read in coordinate data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/city_long_lat.csv"
spark.sparkContext.addFile(url)
coord_df = spark.read.csv(SparkFiles.get("city_long_lat.csv"), sep=",", header=True, inferSchema=True)
coord_df.show()

+--------------+-----------+----------+-----------+
|     city_name|county_name|       lat|       long|
+--------------+-----------+----------+-----------+
|      Shabbona|     DeKalb|41.7680851|-88.8770311|
|    La Prairie|      Adams|40.1478242| -91.002913|
|     New Baden|    Clinton|38.5350488|-89.7006499|
|       Naplate|   La Salle|41.3336449|-88.8781334|
|     Long Lake|       Lake|42.3708546|-88.1275828|
|   Ellis Grove|   Randolph|38.0089404|-89.9059408|
|          Hord|       Clay|38.8844898|-88.5211595|
|   Bunker Hill|   Macoupin|39.0428239| -89.951769|
|     Stockland|   Iroquois|40.6144809| -87.592803|
|     Martinton|   Iroquois|40.9158671|-87.7253124|
|        Altona|       Knox|41.1147594|-90.1645637|
|        Aurora|       Kane|41.7605849|-88.3200715|
|Highland Hills|     DuPage|41.8491966| -88.004509|
|    Wilmington|     Greene|39.4806831|-90.4867659|
| Monroe Center|       Ogle|42.0983583| -89.000655|
|          Akin|   Franklin|37.9881043|-88.7475606|
|    Tower L

In [11]:
# Convert dataframes into Pandas
zillow_df = zillow_df.toPandas()
income_df = income_df.toPandas()
acs_df = acs_df.toPandas()
crime_df = crime_df.toPandas()
city_df = city_df.toPandas()
coord_df = coord_df.toPandas()

# Cleaning Income Data

In [13]:
income_df

Zipcodes Median income (dollars)
0        52761                   55132
1        60002                   82692
2        60004                   94096
3        60005                   78109
4        60007                   79590
...        ...                     ...
1288     62996                  34,063
1289     62997                  56,607
1290     62998                       -
1291     62999                  43,672
1292     63673                  47,803

[1293 rows x 2 columns]

In [14]:
#Rename columns from "Zipcodes" to "zipcode" and "Median income (dollars)" to "med_income" to better fit schemas
#and more easily merge with other tables on "zipcode" as the primary key
income_df = income_df.rename(columns = {"Median income (dollars)":"med_income", "Zipcodes":"zipcode"})

#Remove the commas med_income column to more easily convert to numeric
income_df = income_df.replace(',','', regex=True)

#Remove blank rows with "-" to more easily convert to numeric
income_df = income_df[income_df.med_income != "-"]

#Remove other null values
income_df = income_df.dropna()

#Remove any potential duplicate zipcodes
income_df = income_df.drop_duplicates(subset=['zipcode'])

In [15]:
#Convert median income column to a numeric dtype
income_df[["med_income"]] = income_df[["med_income"]].apply(pd.to_numeric)
income_df

zipcode  med_income
0       52761       55132
1       60002       82692
2       60004       94096
3       60005       78109
4       60007       79590
...       ...         ...
1287    62995       45679
1288    62996       34063
1289    62997       56607
1291    62999       43672
1292    63673       47803

[1254 rows x 2 columns]

# Cleaning Demographic Data



In [16]:
acs_df

zipcode total_pop   male female  ... race_api race_oth eth_hisp eth_nonhisp
0       52761     30619  15205  15414  ...       11      603     5480       25139
1       60002     24066  12076  11990  ...       31      402     1662       22404
2       60004     50915  24193  26722  ...      116     1597     4013       46902
3       60005     29539  15040  14499  ...        0     1089     3163       26376
4       60007     33420  16504  16916  ...       24     1500     4020       29400
...       ...       ...    ...    ...  ...      ...      ...      ...         ...
1380    62996       464    205    259  ...        0        0        0         464
1381    62997       598    314    284  ...        0        0        0         598
1382    62998       323    167    156  ...        0        0        0         323
1383    62999     1,794    915    879  ...        0       29       34       1,760
1384    63673     1,615    778    837  ...        0        0        3       1,612

[1385 rows x 13 columns]

In [17]:
#Remove any commas to more easily convert to numeric
acs_df = acs_df.replace(',','', regex=True)

#Remove rows with "-" in the place of actual data to more easily convert to numeric
acs_df = acs_df[acs_df != "-"]

In [18]:
#Convert all columns to a numeric dtype
acs_df = acs_df.apply(pd.to_numeric)
acs_df

zipcode  total_pop   male  ...  race_oth  eth_hisp  eth_nonhisp
0       52761      30619  15205  ...       603      5480        25139
1       60002      24066  12076  ...       402      1662        22404
2       60004      50915  24193  ...      1597      4013        46902
3       60005      29539  15040  ...      1089      3163        26376
4       60007      33420  16504  ...      1500      4020        29400
...       ...        ...    ...  ...       ...       ...          ...
1380    62996        464    205  ...         0         0          464
1381    62997        598    314  ...         0         0          598
1382    62998        323    167  ...         0         0          323
1383    62999       1794    915  ...        29        34         1760
1384    63673       1615    778  ...         0         3         1612

[1385 rows x 13 columns]

# Cleaning Crime Data

In [19]:
crime_df

_c0 vio_crime  murder_mansl  ... lar_theft car_theft arson
0       Addison        61             1  ...       361        38   4.0
1        Albany         1             0  ...         4         0   0.0
2         Aledo         0             0  ...        23         0   0.0
3        Alexis         0             0  ...         4         4   0.0
4     Algonquin        27             0  ...       243         4   0.0
..          ...       ...           ...  ...       ...       ...   ...
516   Woodstock        24             0  ...       266         5   0.0
517       Worth        26             0  ...        92        10   1.0
518  Yates City         1             0  ...         7         0   0.0
519   Yorkville        18             0  ...       178         5   2.0
520        Zion       148             4  ...       553        39   4.0

[521 rows x 11 columns]

In [20]:
#rename city column for clarity
crime_df = crime_df.rename(columns = {"_c0": "city"})

#Remove the commas to more easily convert to numeric
crime_df = crime_df.replace(',','', regex=True)

In [21]:
#Convert all columns (except "city") to a numeric dtype
crime_df[["vio_crime", "murder_mansl", "rape", "robbery", "agg_assault", "prop_crime", "burglary", "lar_theft", "car_theft", "arson"]] = crime_df[["vio_crime", "murder_mansl", "rape", "robbery", "agg_assault", "prop_crime", "burglary", "lar_theft", "car_theft", "arson"]].apply(pd.to_numeric)
crime_df

city  vio_crime  murder_mansl  ...  lar_theft  car_theft  arson
0       Addison       61.0             1  ...      361.0       38.0    4.0
1        Albany        1.0             0  ...        4.0        0.0    0.0
2         Aledo        0.0             0  ...       23.0        0.0    0.0
3        Alexis        0.0             0  ...        4.0        4.0    0.0
4     Algonquin       27.0             0  ...      243.0        4.0    0.0
..          ...        ...           ...  ...        ...        ...    ...
516   Woodstock       24.0             0  ...      266.0        5.0    0.0
517       Worth       26.0             0  ...       92.0       10.0    1.0
518  Yates City        1.0             0  ...        7.0        0.0    0.0
519   Yorkville       18.0             0  ...      178.0        5.0    2.0
520        Zion      148.0             4  ...      553.0       39.0    4.0

[521 rows x 11 columns]

# Cleaning City Data

In [22]:
city_df

Zip Code               City
0        60001              Alden
1        60002            Antioch
2        60004  Arlington Heights
3        60005  Arlington Heights
4        60006  Arlington Heights
...        ...                ...
1582     62995             Vienna
1583     62996        Villa Ridge
1584     62997        Willisville
1585     62998          Wolf Lake
1586     62999            Zeigler

[1587 rows x 2 columns]

In [23]:
# Rename column names to fit schema and better match the columns of other tables that will be merged for clarity
city_df = city_df.rename(columns = {"Zip Code": "zipcode", "City": "city"})
city_df

zipcode               city
0       60001              Alden
1       60002            Antioch
2       60004  Arlington Heights
3       60005  Arlington Heights
4       60006  Arlington Heights
...       ...                ...
1582    62995             Vienna
1583    62996        Villa Ridge
1584    62997        Willisville
1585    62998          Wolf Lake
1586    62999            Zeigler

[1587 rows x 2 columns]

# Load Clean Tables into PGAdmin

In [24]:
#Convert dataframes back into spark
zillow_df = spark.createDataFrame(zillow_df)
income_df = spark.createDataFrame(income_df)
acs_df = spark.createDataFrame(acs_df)
crime_df = spark.createDataFrame(crime_df)
city_df = spark.createDataFrame(city_df)
coord_df = spark.createDataFrame(coord_df)

In [25]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to zillow_data table in RDS
zillow_df.write.jdbc(url=jdbc_url, table='zhvi_data', mode=mode, properties=config)

In [ ]:
# Write DataFrame to income_data table in RDS
income_df.write.jdbc(url=jdbc_url, table='income_data', mode=mode, properties=config)

In [ ]:
# Write DataFrame to acs_data table in RDS
acs_df.write.jdbc(url=jdbc_url, table='acs_data', mode=mode, properties=config)

In [ ]:
# Write DataFrame to crime_data table in RDS
crime_df.write.jdbc(url=jdbc_url, table='crime_data', mode=mode, properties=config)

In [ ]:
# Write DataFrame to city_data table in RDS
city_df.write.jdbc(url=jdbc_url, table='city_data', mode=mode, properties=config)

In [ ]:
# Write DataFrame to coordinates table in RDS
coord_df.write.jdbc(url=jdbc_url, table='coord_data', mode=mode, properties=config)